In [2]:
import chassisml
from xgboost import XGBRegressor
import pandas as pd
from io import StringIO
import getpass

## Enter credentials
Dockerhub creds and Modzy API Key

In [3]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

## Prepare context dict
Initialize anything here that should persist across inference runs

In [4]:
model = XGBRegressor()
model.load_model('examples/modzy/model_xgboost.txt')

# This will be passed to Chassis:
context = {
    "model": model
}

## Write process function

* Must take bytes and context dict as input
* Preprocess bytes, run inference, postprocess model output, return results

In [6]:
def process(input_bytes,context):
    
    data = StringIO(str(input_bytes,'utf-8')) 
    test_df = pd.read_csv(data)
    raw_preds = context['model'].predict(test_df)
    inference_result = {"predictions": list((raw_preds*100000).astype('float'))}
    
    return inference_result

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [7]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `context` dict containing all variables which should be loaded once and persist across inferences
* Requires `process_fn` defined above

In [8]:
# create Chassis model
chassis_model = chassis_client.create_model(context=context,process_fn=process)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = 'examples/modzy/housing_test.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"predictions":[183435.8811378479,131575.66785812378,128041.79191589355,121437.38269805908,114848.81639480591]}'


## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [9]:
response = chassis_model.publish(model_name="Cal Housing XGBoost",model_version="0.0.1",
                     registry_user=dockerhub_user,registry_pass=dockerhub_pass,
                     modzy_sample_input_path=sample_filepath,
                     modzy_api_key=modzy_api_key)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


## Run sample job using Modzy SDK
Submit inference job to our newly-deploy model running on Modzy

In [ ]:
from modzy import ApiClient

client = ApiClient(base_url='https://integration.modzy.engineering/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)